# Gráficos

Dataset

- Histograma cantidad ratings, ver como hacer con la cola
- Histograma de valores de ratings

Resultados

- Coverage (https://towardsdatascience.com/evaluation-metrics-for-recommender-systems-df56c6611093)


- Métricas
  - mark y mapk
  - personalización

In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import result_io

result_io.write_results(result_io.NAME_COLLABORATIVE, 
    [["1", "2", "3"], ["4"], ["5"]],
    [["1", "2", "3"], ["4"], ["5"]],
)

result_io.read_results(result_io.NAME_COLLABORATIVE)

([['1', '2', '3'], ['4'], ['5']], [['1', '2', '3'], ['4'], ['5']])